In [ ]:
import warnings
from dotenv import load_dotenv

from core.data_sources import CLOBDataSource

warnings.filterwarnings('ignore')
load_dotenv()

CONNECTOR_NAME = "binance"
TRADING_PAIR = "USDT-BRL"
INTERVAL = "1m"

clob = CLOBDataSource()
clob.load_candles_cache(trading_pair=TRADING_PAIR, interval=INTERVAL, connector_name=CONNECTOR_NAME)

candles = clob.get_candles_from_cache(CONNECTOR_NAME, TRADING_PAIR, INTERVAL)

In [ ]:
import pandas as pd
# Filter last 10 days
candles.data = candles.data[candles.data.index >= (pd.Timestamp.now() - pd.Timedelta(days=10))]

df = candles.data

df["buy_volume"] = df["taker_buy_base_volume"]
df["sell_volume"] = df["volume"] - df["buy_volume"]

grouped = df.groupby(df.index.date).agg(
    {
        "open": "first",
        "high": "max",
        "low": "min",
        "close": "last",
        "volume": "sum",
        "buy_volume": "sum",
        "sell_volume": "sum",
    }
).reset_index().rename(columns={"index": "date"})

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

def format_volume(value):
    if value >= 1e9:
        return f"{value/1e9:.1f}B"
    elif value >= 1e6:
        return f"{value/1e6:.1f}M"
    elif value >= 1e3:
        return f"{value/1e3:.1f}K"
    else:
        return f"{value:.1f}"

# Calculate net volume and cumulative net volume
grouped["net_volume"] = grouped["buy_volume"] - grouped["sell_volume"]
grouped["cumulative_net"] = grouped["net_volume"].cumsum()

# Calculate price ranges
today = pd.Timestamp.now().normalize()
today_data = candles.data[candles.data.index.date == today.date()]
last_7_days = candles.data[candles.data.index >= (candles.data.index.max() - pd.Timedelta(days=7))]

# Today's range
if not today_data.empty:
    today_high = today_data['high'].max()
    today_low = today_data['low'].min()
    today_range_pct = ((today_high - today_low) / today_low) * 100
else:
    today_high = today_low = today_range_pct = 0

# 7-day range
week_high = last_7_days['high'].max()
week_low = last_7_days['low'].min()
week_range_pct = ((week_high - week_low) / week_low) * 100

# Create figure with 2 rows: candles on top, volume analysis below
fig = make_subplots(
    rows=2, cols=1,
    row_heights=[0.6, 0.4],
    specs=[[{"secondary_y": False}],
           [{"secondary_y": True}]],
    subplot_titles=(f"{TRADING_PAIR} Price (1m candles)", "Volume Analysis"),
    vertical_spacing=0.1
)

# Add candlestick chart to the top subplot
candles_trace = candles.candles_trace()
fig.add_trace(candles_trace, row=1, col=1)

# Add 7-day range box
fig.add_shape(
    type="rect",
    x0=last_7_days.index.min(),
    x1=last_7_days.index.max(),
    y0=week_low,
    y1=week_high,
    line=dict(color="#8B5CF6", width=2, dash="dash"),
    fillcolor="rgba(139, 92, 246, 0.1)",
    row=1, col=1
)

# Add 7-day range annotation
fig.add_annotation(
    x=last_7_days.index.max(),
    y=week_high,
    text=f"7D Range: {week_range_pct:.2f}%",
    showarrow=False,
    bgcolor="#8B5CF6",
    font=dict(color="white", size=10),
    xanchor="right",
    yanchor="bottom",
    row=1, col=1
)

# Add today's range box if we have today's data
if not today_data.empty:
    fig.add_shape(
        type="rect",
        x0=today_data.index.min(),
        x1=today_data.index.max(),
        y0=today_low,
        y1=today_high,
        line=dict(color="#F59E0B", width=2),
        fillcolor="rgba(245, 158, 11, 0.2)",
        row=1, col=1
    )
    
    # Add today's range annotation
    fig.add_annotation(
        x=today_data.index.max() if not today_data.empty else last_7_days.index.max(),
        y=today_high,
        text=f"Today: {today_range_pct:.2f}%",
        showarrow=False,
        bgcolor="#F59E0B",
        font=dict(color="white", size=10),
        xanchor="right",
        yanchor="top",
        row=1, col=1
    )

# Add buy volume bar to bottom subplot
fig.add_trace(go.Bar(
    x=grouped["date"],
    y=grouped["buy_volume"],
    name="Buy Volume",
    marker_color="#10B981",  # Emerald green
    text=[format_volume(v) for v in grouped["buy_volume"]],
    textposition="inside",
    offsetgroup=1
), row=2, col=1, secondary_y=False)

# Add sell volume bar to bottom subplot
fig.add_trace(go.Bar(
    x=grouped["date"],
    y=grouped["sell_volume"],
    name="Sell Volume",
    marker_color="#EF4444",  # Coral red
    text=[format_volume(v) for v in grouped["sell_volume"]],
    textposition="inside",
    offsetgroup=1
), row=2, col=1, secondary_y=False)

# Add cumulative net volume line to bottom subplot
fig.add_trace(go.Scatter(
    x=grouped["date"],
    y=grouped["cumulative_net"],
    name="Cumulative Net Volume",
    mode="lines+markers",
    line=dict(color="#3B82F6", width=2),  # Bright blue
    marker=dict(size=8),
    text=[format_volume(v) for v in grouped["cumulative_net"]],
    textposition="top center"
), row=2, col=1, secondary_y=True)

# Update layout
fig.update_layout(
    title=f"Market Analysis: {TRADING_PAIR} on {CONNECTOR_NAME}<br><sup>Total Volume: {format_volume(grouped['volume'].sum())} | Net Volume: {format_volume(grouped['net_volume'].sum())}</sup>",
    hovermode="x unified",
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.15,
        xanchor="center",
        x=0.5
    ),
    height=900,
    showlegend=True,
    barmode="stack",
    xaxis_rangeslider_visible=False,
    xaxis2_rangeslider_visible=False
)

# Update x-axes
fig.update_xaxes(title_text="", row=1, col=1, rangeslider_visible=False)
fig.update_xaxes(title_text="Date", row=2, col=1, rangeslider_visible=False)

# Update y-axes
fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="Volume", row=2, col=1, secondary_y=False)
fig.update_yaxes(title_text="Cumulative Net", row=2, col=1, secondary_y=True)

fig.show()

In [ ]:
# Save chart for Telegram
import os
from core.notifiers import NotificationManager
import plotly.io as pio

# Use temporary directory like in telegram_screener_report.ipynb
chart_png_path = '/tmp/brigado_market_analysis.png'

# Generate filename with timestamp
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

print("📊 Generating PNG chart for Telegram...")

try:
    img_bytes = pio.to_image(fig, format="png", width=1200, height=900, scale=1)
    with open(chart_png_path, 'wb') as f:
        f.write(img_bytes)
    print(f"✅ PNG chart saved: {chart_png_path}")
    chart_success = True
except Exception as e:
    print(f"❌ PNG generation failed: {e}")
    chart_success = False

# Create summary message for Telegram
summary_message = f"""📊 <b>Market Analysis: {TRADING_PAIR} on {CONNECTOR_NAME}</b>

📈 <b>7-Day Range:</b> {week_range_pct:.2f}%
📅 <b>Today's Range:</b> {today_range_pct:.2f}% {'(No data)' if today_range_pct == 0 else ''}

💰 <b>Volume Summary:</b>
• Total Volume: {format_volume(grouped['volume'].sum())}
• Net Volume: {format_volume(grouped['net_volume'].sum())}
• Buy Pressure: {(grouped['buy_volume'].sum() / grouped['volume'].sum() * 100):.1f}%

🕐 <i>Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</i>
"""

print("\nTelegram message:")
print(summary_message)

# Initialize notification manager and send via Telegram
notification_manager = NotificationManager()
telegram_notifier = notification_manager.get_notifier('telegram')

if telegram_notifier and chart_success:
    print(f"\n📤 Sending PNG chart and message via Telegram...")
    
    try:
        # Send photo with caption
        send_success = await telegram_notifier.send_photo(
            chart_png_path,
            caption=summary_message
        )
        
        print(f"📊 Chart: {'✅ Sent successfully!' if send_success else '❌ Failed to send'}")
        
        if send_success:
            print("🎉 Market analysis delivered to Telegram!")
        else:
            print("⚠️ Failed to send chart - check Telegram configuration")
            
    except Exception as e:
        print(f"❌ Error sending to Telegram: {e}")
        print(f"💾 Chart saved locally at: {chart_png_path}")
        
else:
    print("⚠️ Telegram notifier not configured or PNG generation failed")
    if chart_success:
        print(f"💾 Chart saved locally at: {chart_png_path}")
    print("💡 Configure Telegram in .env to send automatically")